# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 
CNN Model:
I don't touch CNN from default
RNN Model:
My model is very simple.
I have maden the input with features and captions except the last word <end>.
So, I made captions embedding first, then concatenates features and caption embeddings using torch.cat.
And, I removed the last with torch.index_select. Actually I selected all the columns except the last column corresponding to the last words of the captions.
I left several comments on my model.py.
Then, I input the input tensor I made to LSTM to make LSTM output and update the hidden states.
Then, I made the final outputs with nn.Linear and LSTM output.

batch_size(128): I wanted to make bigger batch_size because usuable GPU time is limited although smaller batch_size makes RNN models much more accurate. 

vocab_threshold(8): I thought that the words that appeared frequently were important when I train my model. So I used 8 as vocab_threshold than the default setting(4). 

embed_size(256), hidden_size(512): I used the default setting. I don't have much time to try several values becuase of limited resource. 

num_epochs(1): I think more than 3 is better but I don't have enough GPU time. 


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** I used default because it was optional.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** My model trains the parameters of encoder only. Decoder works just like "Decoder". A decoder replaces numbers with corresponding words. Decoder doesn't need to be trained.

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** I select Adam as my optimizer. Adam makes parameters to converge quickly. I used it at project 1 also.

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
#!pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 128          # batch size
vocab_threshold = 8        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 256           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 1             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.68s)
creating index...


  0%|          | 689/414113 [00:00<01:00, 6887.23it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:54<00:00, 7619.29it/s]


Init RNN


In [11]:
print(len(data_loader.dataset.caption_lengths))
print(data_loader.batch_sampler.batch_size)

414113
128


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [12]:
#torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
#torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

In [13]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
"""
# Comment out because AWS cannot resolve metadata.google.internal
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})
"""
for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            """
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
            """
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        print(outputs.shape)
        print(captions.shape)
        print(outputs.view(-1, vocab_size).shape, captions.view(-1).shape)
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

torch.Size([128, 11, 7073])
torch.Size([128, 11])
torch.Size([1408, 7073]) torch.Size([1408])
Epoch [1/1], Step [1/3236], Loss: 8.8715, Perplexity: 7125.6290torch.Size([128, 13, 7073])
torch.Size([128, 13])
torch.Size([1664, 7073]) torch.Size([1664])
Epoch [1/1], Step [2/3236], Loss: 8.6421, Perplexity: 5665.2918torch.Size([128, 18, 7073])
torch.Size([128, 18])
torch.Size([2304, 7073]) torch.Size([2304])
Epoch [1/1], Step [3/3236], Loss: 8.4880, Perplexity: 4856.2780torch.Size([128, 12, 7073])
torch.Size([128, 12])
torch.Size([1536, 7073]) torch.Size([1536])
Epoch [1/1], Step [4/3236], Loss: 7.9614, Perplexity: 2868.0678torch.Size([128, 14, 7073])
torch.Size([128, 14])
torch.Size([1792, 7073]) torch.Size([1792])
Epoch [1/1], Step [5/3236], Loss: 7.7407, Perplexity: 2300.0254torch.Size([128, 13, 7073])
torch.Size([128, 13])
torch.Size([1664, 7073]) torch.Size([1664])
Epoch [1/1], Step [6/3236], Loss: 7.2279, Perplexity: 1377.2917torch.Size([128, 11, 7073])
torch.Size([128, 11])
torch.Si

Epoch [1/1], Step [53/3236], Loss: 4.4706, Perplexity: 87.4058torch.Size([128, 16, 7073])
torch.Size([128, 16])
torch.Size([2048, 7073]) torch.Size([2048])
Epoch [1/1], Step [54/3236], Loss: 4.6777, Perplexity: 107.5213torch.Size([128, 11, 7073])
torch.Size([128, 11])
torch.Size([1408, 7073]) torch.Size([1408])
Epoch [1/1], Step [55/3236], Loss: 4.4445, Perplexity: 85.1610torch.Size([128, 13, 7073])
torch.Size([128, 13])
torch.Size([1664, 7073]) torch.Size([1664])
Epoch [1/1], Step [56/3236], Loss: 4.4050, Perplexity: 81.8599torch.Size([128, 16, 7073])
torch.Size([128, 16])
torch.Size([2048, 7073]) torch.Size([2048])
Epoch [1/1], Step [57/3236], Loss: 4.6208, Perplexity: 101.5760torch.Size([128, 14, 7073])
torch.Size([128, 14])
torch.Size([1792, 7073]) torch.Size([1792])
Epoch [1/1], Step [58/3236], Loss: 4.4021, Perplexity: 81.6204torch.Size([128, 11, 7073])
torch.Size([128, 11])
torch.Size([1408, 7073]) torch.Size([1408])
Epoch [1/1], Step [59/3236], Loss: 4.4598, Perplexity: 86.4734

Epoch [1/1], Step [106/3236], Loss: 4.1862, Perplexity: 65.7702torch.Size([128, 15, 7073])
torch.Size([128, 15])
torch.Size([1920, 7073]) torch.Size([1920])
Epoch [1/1], Step [107/3236], Loss: 4.1975, Perplexity: 66.5217torch.Size([128, 17, 7073])
torch.Size([128, 17])
torch.Size([2176, 7073]) torch.Size([2176])
Epoch [1/1], Step [108/3236], Loss: 4.3988, Perplexity: 81.3551torch.Size([128, 14, 7073])
torch.Size([128, 14])
torch.Size([1792, 7073]) torch.Size([1792])
Epoch [1/1], Step [109/3236], Loss: 4.0425, Perplexity: 56.9679torch.Size([128, 13, 7073])
torch.Size([128, 13])
torch.Size([1664, 7073]) torch.Size([1664])
Epoch [1/1], Step [110/3236], Loss: 4.0609, Perplexity: 58.0253torch.Size([128, 21, 7073])
torch.Size([128, 21])
torch.Size([2688, 7073]) torch.Size([2688])
Epoch [1/1], Step [111/3236], Loss: 4.7048, Perplexity: 110.4708torch.Size([128, 10, 7073])
torch.Size([128, 10])
torch.Size([1280, 7073]) torch.Size([1280])
Epoch [1/1], Step [112/3236], Loss: 4.3766, Perplexity: 7

Epoch [1/1], Step [158/3236], Loss: 4.2994, Perplexity: 73.6528torch.Size([128, 12, 7073])
torch.Size([128, 12])
torch.Size([1536, 7073]) torch.Size([1536])
Epoch [1/1], Step [159/3236], Loss: 3.8860, Perplexity: 48.7179torch.Size([128, 20, 7073])
torch.Size([128, 20])
torch.Size([2560, 7073]) torch.Size([2560])
Epoch [1/1], Step [160/3236], Loss: 4.3920, Perplexity: 80.8013torch.Size([128, 12, 7073])
torch.Size([128, 12])
torch.Size([1536, 7073]) torch.Size([1536])
Epoch [1/1], Step [161/3236], Loss: 3.7827, Perplexity: 43.9341torch.Size([128, 14, 7073])
torch.Size([128, 14])
torch.Size([1792, 7073]) torch.Size([1792])
Epoch [1/1], Step [162/3236], Loss: 3.6809, Perplexity: 39.6801torch.Size([128, 14, 7073])
torch.Size([128, 14])
torch.Size([1792, 7073]) torch.Size([1792])
Epoch [1/1], Step [163/3236], Loss: 3.5985, Perplexity: 36.5450torch.Size([128, 12, 7073])
torch.Size([128, 12])
torch.Size([1536, 7073]) torch.Size([1536])
Epoch [1/1], Step [164/3236], Loss: 3.6596, Perplexity: 38

Epoch [1/1], Step [210/3236], Loss: 3.6321, Perplexity: 37.7920torch.Size([128, 16, 7073])
torch.Size([128, 16])
torch.Size([2048, 7073]) torch.Size([2048])
Epoch [1/1], Step [211/3236], Loss: 3.9464, Perplexity: 51.7462torch.Size([128, 11, 7073])
torch.Size([128, 11])
torch.Size([1408, 7073]) torch.Size([1408])
Epoch [1/1], Step [212/3236], Loss: 3.6516, Perplexity: 38.5373torch.Size([128, 15, 7073])
torch.Size([128, 15])
torch.Size([1920, 7073]) torch.Size([1920])
Epoch [1/1], Step [213/3236], Loss: 3.6031, Perplexity: 36.7128torch.Size([128, 13, 7073])
torch.Size([128, 13])
torch.Size([1664, 7073]) torch.Size([1664])
Epoch [1/1], Step [214/3236], Loss: 3.5226, Perplexity: 33.8732torch.Size([128, 11, 7073])
torch.Size([128, 11])
torch.Size([1408, 7073]) torch.Size([1408])
Epoch [1/1], Step [215/3236], Loss: 3.6592, Perplexity: 38.8288torch.Size([128, 15, 7073])
torch.Size([128, 15])
torch.Size([1920, 7073]) torch.Size([1920])
Epoch [1/1], Step [216/3236], Loss: 3.6726, Perplexity: 39

...

Epoch [1/1], Step [3018/3236], Loss: 2.2258, Perplexity: 9.2607torch.Size([128, 29, 7073])
torch.Size([128, 29])
torch.Size([3712, 7073]) torch.Size([3712])
Epoch [1/1], Step [3019/3236], Loss: 3.5008, Perplexity: 33.1430torch.Size([128, 14, 7073])
torch.Size([128, 14])
torch.Size([1792, 7073]) torch.Size([1792])
Epoch [1/1], Step [3020/3236], Loss: 2.1133, Perplexity: 8.2758torch.Size([128, 12, 7073])
torch.Size([128, 12])
torch.Size([1536, 7073]) torch.Size([1536])
Epoch [1/1], Step [3021/3236], Loss: 2.2542, Perplexity: 9.5279torch.Size([128, 16, 7073])
torch.Size([128, 16])
torch.Size([2048, 7073]) torch.Size([2048])
Epoch [1/1], Step [3022/3236], Loss: 2.5289, Perplexity: 12.5402torch.Size([128, 12, 7073])
torch.Size([128, 12])
torch.Size([1536, 7073]) torch.Size([1536])
Epoch [1/1], Step [3023/3236], Loss: 2.2597, Perplexity: 9.5800torch.Size([128, 12, 7073])
torch.Size([128, 12])
torch.Size([1536, 7073]) torch.Size([1536])
Epoch [1/1], Step [3024/3236], Loss: 2.2090, Perplexity:

Epoch [1/1], Step [3070/3236], Loss: 2.1441, Perplexity: 8.5342torch.Size([128, 10, 7073])
torch.Size([128, 10])
torch.Size([1280, 7073]) torch.Size([1280])
Epoch [1/1], Step [3071/3236], Loss: 2.5479, Perplexity: 12.7802torch.Size([128, 10, 7073])
torch.Size([128, 10])
torch.Size([1280, 7073]) torch.Size([1280])
Epoch [1/1], Step [3072/3236], Loss: 2.4346, Perplexity: 11.4116torch.Size([128, 11, 7073])
torch.Size([128, 11])
torch.Size([1408, 7073]) torch.Size([1408])
Epoch [1/1], Step [3073/3236], Loss: 2.1891, Perplexity: 8.9269torch.Size([128, 10, 7073])
torch.Size([128, 10])
torch.Size([1280, 7073]) torch.Size([1280])
Epoch [1/1], Step [3074/3236], Loss: 2.5121, Perplexity: 12.3308torch.Size([128, 14, 7073])
torch.Size([128, 14])
torch.Size([1792, 7073]) torch.Size([1792])
Epoch [1/1], Step [3075/3236], Loss: 2.3365, Perplexity: 10.3447torch.Size([128, 11, 7073])
torch.Size([128, 11])
torch.Size([1408, 7073]) torch.Size([1408])
Epoch [1/1], Step [3076/3236], Loss: 2.2531, Perplexit

Epoch [1/1], Step [3122/3236], Loss: 2.5451, Perplexity: 12.7444torch.Size([128, 13, 7073])
torch.Size([128, 13])
torch.Size([1664, 7073]) torch.Size([1664])
Epoch [1/1], Step [3123/3236], Loss: 2.1825, Perplexity: 8.8689torch.Size([128, 12, 7073])
torch.Size([128, 12])
torch.Size([1536, 7073]) torch.Size([1536])
Epoch [1/1], Step [3124/3236], Loss: 2.1938, Perplexity: 8.9690torch.Size([128, 16, 7073])
torch.Size([128, 16])
torch.Size([2048, 7073]) torch.Size([2048])
Epoch [1/1], Step [3125/3236], Loss: 2.2677, Perplexity: 9.6567torch.Size([128, 12, 7073])
torch.Size([128, 12])
torch.Size([1536, 7073]) torch.Size([1536])
Epoch [1/1], Step [3126/3236], Loss: 2.1804, Perplexity: 8.8496torch.Size([128, 12, 7073])
torch.Size([128, 12])
torch.Size([1536, 7073]) torch.Size([1536])
Epoch [1/1], Step [3127/3236], Loss: 2.0815, Perplexity: 8.0168torch.Size([128, 18, 7073])
torch.Size([128, 18])
torch.Size([2304, 7073]) torch.Size([2304])
Epoch [1/1], Step [3128/3236], Loss: 2.6009, Perplexity: 

Epoch [1/1], Step [3174/3236], Loss: 2.2125, Perplexity: 9.1388torch.Size([128, 18, 7073])
torch.Size([128, 18])
torch.Size([2304, 7073]) torch.Size([2304])
Epoch [1/1], Step [3175/3236], Loss: 2.6561, Perplexity: 14.2402torch.Size([128, 10, 7073])
torch.Size([128, 10])
torch.Size([1280, 7073]) torch.Size([1280])
Epoch [1/1], Step [3176/3236], Loss: 2.4683, Perplexity: 11.8028torch.Size([128, 15, 7073])
torch.Size([128, 15])
torch.Size([1920, 7073]) torch.Size([1920])
Epoch [1/1], Step [3177/3236], Loss: 2.3069, Perplexity: 10.0437torch.Size([128, 12, 7073])
torch.Size([128, 12])
torch.Size([1536, 7073]) torch.Size([1536])
Epoch [1/1], Step [3178/3236], Loss: 1.9419, Perplexity: 6.9716torch.Size([128, 50, 7073])
torch.Size([128, 50])
torch.Size([6400, 7073]) torch.Size([6400])
Epoch [1/1], Step [3179/3236], Loss: 4.3387, Perplexity: 76.6106torch.Size([128, 13, 7073])
torch.Size([128, 13])
torch.Size([1664, 7073]) torch.Size([1664])
Epoch [1/1], Step [3180/3236], Loss: 2.1972, Perplexit

Epoch [1/1], Step [3226/3236], Loss: 2.2157, Perplexity: 9.1682torch.Size([128, 10, 7073])
torch.Size([128, 10])
torch.Size([1280, 7073]) torch.Size([1280])
Epoch [1/1], Step [3227/3236], Loss: 2.3227, Perplexity: 10.2035torch.Size([128, 12, 7073])
torch.Size([128, 12])
torch.Size([1536, 7073]) torch.Size([1536])
Epoch [1/1], Step [3228/3236], Loss: 2.1375, Perplexity: 8.4783torch.Size([128, 13, 7073])
torch.Size([128, 13])
torch.Size([1664, 7073]) torch.Size([1664])
Epoch [1/1], Step [3229/3236], Loss: 2.0938, Perplexity: 8.1160torch.Size([128, 16, 7073])
torch.Size([128, 16])
torch.Size([2048, 7073]) torch.Size([2048])
Epoch [1/1], Step [3230/3236], Loss: 2.3742, Perplexity: 10.7428torch.Size([128, 16, 7073])
torch.Size([128, 16])
torch.Size([2048, 7073]) torch.Size([2048])
Epoch [1/1], Step [3231/3236], Loss: 2.5075, Perplexity: 12.2747torch.Size([128, 12, 7073])
torch.Size([128, 12])
torch.Size([1536, 7073]) torch.Size([1536])
Epoch [1/1], Step [3232/3236], Loss: 2.2495, Perplexity

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [13]:
# (Optional) TODO: Validate your model.